In [169]:
import pandas as pd
import pyodbc
import os
from datetime import datetime
import time
import warnings
import numpy as np

In [170]:

%store -r server
%store -r database
%store -r username
%store -r password
%store -r driver
%store -r azure_container_name
%store -r storage_account_name
%store -r database_table_name
#%store -r sas_connection_string





In [171]:
#container_name = azure_container_name[1]
#connection_string = sas_connection_string

In [172]:
# def transfer_file_to_container(filename, blob_name, container_name, connection_string):
#     # Import the client object from the SDK library
#     from azure.storage.blob import BlobClient

#     # Retrieve the connection string from an environment variable. Note that a
#     # connection string grants all permissions to the caller, making it less
#     # secure than obtaining a BlobClient object using credentials.
#     connection_string = connection_string

#     # Create the client object for the resource identified by the connection
#     # string, indicating also the blob container and the name of the specific
#     # blob we want.
#     blob_client = BlobClient.from_connection_string(
#         connection_string,
#         container_name= container_name,
#         blob_name= blob_name,
#     )

#     # Open a local file and upload its contents to Blob Storage
#     with open(f"./{filename}", "rb") as data:
#         blob_client.upload_blob(data)
#         print(f"Uploaded {filename} to {blob_client.url}")


        
# def move_file_to_container(filename):
#     current_directory = os.getcwd()
#     files_in_directory = os.listdir(current_directory)
    
#     if filename in files_in_directory:
#         print(f"{filename} exists in the current directory")
#         transfer_file_to_container(filename, filename, container_name, connection_string)
        
#     else:
#         print(f"{filename} is not in the current directory")

In [173]:
# Load your data into a DataFrame
try:
    data = pd.read_csv('Completed_Cleaned_Jobberman.csv')  # Replace 'your_data.csv' with your data source
    #data.to_parquet(f"Completed_Cleaned_Jobberman.parquet")
    data["Generic_Job_Name"] = data["Generic_Job_Name"].str.strip("[]").str.strip("''").str.strip('""')  
    #data = data.iloc[15:, :]

except FileNotFoundError as e:
    print(e)
    data = ""


In [174]:
def checking_recorded_column_names(dataframe):
    df = dataframe.copy()
    mask_check = all (df[col].isin(df.columns).sum() for col in df.columns)

    if mask_check:
        mask = df["Salary"].isin(df.columns)
        idx = df.loc[mask, :].index
        print(idx)
        df.drop(idx, inplace = True)
        print(f"{len(idx)} rows dropped because they are recorded column names")
        
    return df

def correct_openai_format(string_data):
    start_index = string_data.find('"content": "') + len('"content": "')
    end_index = string_data.find('"', start_index)
    content = string_data[start_index:end_index]
    
    return content


def columns_check(dframe):
    df = dframe.copy()
    
    try:
        if df["Generic_Job_Name"].str.len().max() > 55:
            df["Generic_Job_Name"] = np.where(df["Generic_Job_Name"].str.contains("<OpenAIObject"), 
                                               df["Generic_Job_Name"].apply(correct_openai_format), df["Generic_Job_Name"])

        if df["Generic_Job_Name"].str.startswith("[").sum() > 0:
            print("remvoing brackets")
            df["Generic_Job_Name"] = df["Generic_Job_Name"].str.replace("[", "")
            df["Generic_Job_Name"] = df["Generic_Job_Name"].str.replace("]", "")
            
        if df["Generic_Job_Name"].str.startswith("'").sum() > 0:
            #df["Generic_Job_Name"] = df["Generic_Job_Name"].str.strip()
            df["Generic_Job_Name"] = df["Generic_Job_Name"].str.strip("''")

            
        if df["Scrape_Time"].dtype == "O":
            try:
                df["Scrape_Time"] = pd.to_datetime(df["Scrape_Time"], format='mixed', dayfirst= True)
                
            except ValueError as e:
                print(f"Encountered an error.\nPython error: {e}")
                df = checking_recorded_column_names(df)
                
            

        if df["Day_Posted"].dtype == "O":
            df["Day_Posted"] = pd.to_datetime(df["Day_Posted"])
            
    except Exception as e:
        warnings.warn("column check could not be completed")
        print(f"Python error: {e}")
              
    
    return df
        

        

In [175]:
data = columns_check(data)

Encountered an error.
Python error: Unknown datetime string format, unable to parse: Scrape_Time, at position 20
Index([20, 41, 62, 83, 104, 125, 146, 167], dtype='int64')
8 rows dropped because they are recorded column names


In [176]:
# k = data [data["Generic_Job_Name"].str.len()> 55]
# k["Generic_Job_Name"].apply(correct_openai_format).str.len().max()

data[data["Generic_Job_Name"].str.contains("<OpenAIObject")]

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url


In [177]:
try:
    rowed_data = tuple(data.values.tolist())

except (AttributeError, Exception) as e:
    print(f"No cleaned CSV file in the local storage.\nPython error:{e}")
    rowed_data = ""

In [178]:
table_name = database_table_name

In [179]:
table_name

'[dbo].[Final_template_df]'

In [180]:
max_retries = 3

# Function to establish a database connection
def establish_connection():
    for attempt in range(max_retries):
        try:
            cnxn = pyodbc.connect('DRIVER=' + driver + 
                                 ';SERVER=' + server + 
                                 ';DATABASE=' + database + 
                                 ';UID=' + username + 
                                 ';PWD=' + password)

            cursor = cnxn.cursor()
            print('Connection established')
            return cnxn, cursor
        except pyodbc.Error as e:
            print(f'Connection attempt {attempt + 1} failed: {str(e)}')
            if attempt < max_retries - 1:
                # Sleep for a few seconds before retrying
                time.sleep(5)
    
    print('Cannot connect to SQL server after retries.')
    return None, None

# Attempt to establish a connection

if isinstance(rowed_data, str):
    print("Data is a string. This means there is no cleaned CSV file")
else:
    cnxn, cursor = establish_connection()

    if cnxn:
        insert_sql = f"""INSERT INTO {table_name} (Job_Name, Generic_Job_Name, Company_Name, Industry, Location,
            Job_Type, Job_Summary, Education_Requirements,
            Experience_Level_Requirements, Experience_Length_Requirements,
            Job_Description, Salary, Salary_Range, Date_Posted,
            Day_Posted, Scrape_Time, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

        try:
            successful_inserts = 0
            unsuccessful_inserts_list =[]
            # Execute the INSERT statement for each set of data
            for i, row in enumerate(rowed_data):
                try:
                    cursor.execute(insert_sql, row)
                    cnxn.commit()
                    successful_inserts += 1
                    print(f"Row {i} inserted successfully")
                except pyodbc.Error as e:
                    print(f'Error inserting row {i}: {str(e)}')
                    cnxn.rollback()
                    unsuccessful_inserts_list.append(row)

            print(f'Total successful insertions: {successful_inserts} out of {len(rowed_data)} rows')
        except pyodbc.Error as e:
            print(f'Error inserting data: {str(e)}')
        finally:
            if len(unsuccessful_inserts_list) > 0:
                unsuccessful_inserts_df = pd.DataFrame(unsuccessful_inserts_list)
                unsuccessful_inserts_df.to_csv("Unsuccessful_database_inserts.csv", index = False)
            cursor.close()
            cnxn.close()

# if isinstance(rowed_data, str):
#     print("Data is a string. This means there is no cleaned CSV file")

# else:
#     cnxn, cursor = establish_connection()





#     if cnxn:
#         insert_sql = f"""INSERT INTO {table_name} (Job_Name, Generic_Job_Name, Company_Name, Industry, Location,
#             Job_Type, Job_Summary, Education_Requirements,
#             Experience_Level_Requirements, Experience_Length_Requirements,
#             Job_Description, Salary, Salary_Range, Date_Posted,
#             Day_Posted, Scrape_Time, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

#         try:
#             # Execute the INSERT statement for each set of data
#             for i, row in enumerate(rowed_data):
#                 cursor.execute(insert_sql, row)
#                 print(f"Row {i} inserted successfully")

#             # Commit the changes to the database
#             cnxn.commit()
#             print('Data inserted successfully.')
#         except pyodbc.Error as e:
#             print(f'Error inserting data: {str(e)}')
#             # Rollback the transaction in case of an error
#             cnxn.rollback()
#         finally:
#             # Close the cursor and the database connection
#             cursor.close()
#             cnxn.close()


Connection attempt 1 failed: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Unable to complete login process due to delay in login response (258)')
Connection established
Row 0 inserted successfully
Row 1 inserted successfully
Row 2 inserted successfully
Row 3 inserted successfully
Row 4 inserted successfully
Row 5 inserted successfully
Row 6 inserted successfully
Row 7 inserted successfully
Row 8 inserted successfully
Row 9 inserted successfully
Row 10 inserted successfully
Row 11 inserted successfully
Row 12 inserted successfully
Row 13 inserted successfully
Row 14 inserted successfully
Row 15 inserted successfully
Row 16 inserted successfully
Row 17 inserted successfully
Row 18 inserted successfully
Row 19 inserted successfully
Row 20 inserted successfully
Row 21 inserted successfull

In [168]:
data

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Plumber,Other,AQUATRIO NIGERIA LTD,Trades & Services,Lagos,Full Time,"This is aquatrio Nigeria Limited, we are the s...",High School (S.S.C.E),Entry level,3 years,Fixing water filter in the company for any of ...,Commission Only,Commission Only,3 days ago,2023-11-06,2023-11-09 20:59:30,https://www.jobberman.com/listings/plumber-q2qjwp
1,Training Manager,Administrative Officer,Noble Guard Security (Noble Group),Consulting & Strategy,Lagos,Full Time,Ready to join our team of security professiona...,HND,Mid level,3 years,Evaluate organizational performance to ensure ...,150000,"150,000 - 250,000",3 days ago,2023-11-06,2023-11-09 20:59:40,https://www.jobberman.com/listings/training-ma...
2,Accountant,Financial Officer,GRAND SWISS HOTEL LTD,"Accounting, Auditing & Finance",Anambra,Full Time,"As a Hotel Accountant, you will play a critica...",Degree,Entry level,1 year,"Financial Reporting: Prepare, review, and anal...",75000,"75,000 - 150,000",3 days ago,2023-11-06,2023-11-09 20:59:59,https://www.jobberman.com/listings/accountant-...
3,Driver,Driver,GRAND SWISS HOTEL LTD,Driver & Transport Services,Anambra,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,5 years,Safely and timely drive Hotel guests to all de...,75000,"75,000 - 150,000",3 days ago,2023-11-06,2023-11-09 21:00:08,https://www.jobberman.com/listings/driver-20ze7g
4,Marketing Executive,Other,Hathor Coast Limited,Marketing & Communications,Delta,Full Time,We are seeking a full-time Marketing Executive...,HND,Entry level,1 year,"HND/BSc in Marketing, Communications, Business...",75000,"Less than 75,000",3 days ago,2023-11-06,2023-11-09 21:00:16,https://www.jobberman.com/listings/marketing-e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,ICT Teacher,Teacher,Loral International Schools,Software & Data,Lagos,Full Time,A Computer Teacher is responsible for planning...,HND,Mid level,3 years,Curriculum Development: Design and develop com...,75000,"75,000 - 150,000",6 days ago,2023-11-03,2023-11-09 21:07:33,https://www.jobberman.com/listings/ict-teacher...
163,Procurement Manager,Inventory Manager/Store Keeper,Tequila Nigeria Limited,Supply Chain & Procurement,Lagos,Full Time,A female professional procurement Manager with...,Degree,Mid level,5 years,Research\npotential vendors.Compare\nand evalu...,250000,"250,000 - 400,000",2 days ago,2023-11-07,2023-11-09 21:07:41,https://www.jobberman.com/listings/procurement...
164,Cross Platform Mobile App Developer,Software Developer/Engineer,Bakkaz Trading and Investments Ltd,Software & Data,Lagos,Full Time,Cross-platform mobile development is an approa...,Degree,Entry level,2 years,At least 1 - 2 Years of Mobile App Development...,150000,"150,000 - 250,000",6 days ago,2023-11-03,2023-11-09 21:07:47,https://www.jobberman.com/listings/cross-platf...
165,Web Developer,Software Developer/Engineer,Bakkaz Trading and Investments Ltd,Software & Data,Lagos,Full Time,Web developers build and maintain websites and...,Degree,Entry level,2 years,Proven working experience in web programmingTo...,150000,"150,000 - 250,000",6 days ago,2023-11-03,2023-11-09 21:07:54,https://www.jobberman.com/listings/web-develop...


In [104]:
# if isinstance(rowed_data, str):
#     print("Data is a string. This means there is no cleaned CSV file")
# else:
#     cnxn, cursor = establish_connection()

#     if cnxn:
#         insert_sql = f"""INSERT INTO {table_name} (Job_Name, Generic_Job_Name, Company_Name, Industry, Location,
#             Job_Type, Job_Summary, Education_Requirements,
#             Experience_Level_Requirements, Experience_Length_Requirements,
#             Job_Description, Salary, Salary_Range, Date_Posted,
#             Day_Posted, Scrape_Time, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

#         try:
#             successful_inserts = 0
#             # Execute the INSERT statement for each set of data
#             for i, row in enumerate(rowed_data):
#                 try:
#                     cursor.execute(insert_sql, row)
#                     cnxn.commit()
#                     successful_inserts += 1
#                     print(f"Row {i} inserted successfully")
#                 except pyodbc.Error as e:
#                     print(f'Error inserting row {i}: {str(e)}')
#                     cnxn.rollback()

#             print(f'Total successful insertions: {successful_inserts} out of {len(rowed_data)} rows')
#         except pyodbc.Error as e:
#             print(f'Error inserting data: {str(e)}')
#         finally:
#             cursor.close()
#             cnxn.close()


In [105]:

# data[data["Generic_Job_Name"].str.len() > 50]["Generic_Job_Name"][42]

In [23]:
# with conn.cursor() as cursor:
#     for index, row in data.iterrows():
#         values = tuple(row)  # Convert the row to a tuple for the parameterized query
#         cursor.execute(insert_query, values)
#     conn.commit()  # Commit the changes


In [24]:
# try:
#     cnxn = pyodbc.connect('DRIVER=' + driver + 
#                       ';SERVER=' + server + 
#                       ';DATABASE=' + database + 
#                       ';UID=' + username + 
#                       ';PWD=' + password)

#     cursor = cnxn.cursor()
#     print('Connection established')
# except:
#     print('Cannot connect to SQL server')

# insert_sql = f"""INSERT INTO {table_name} (Job_Name, Generic_Job_Name, Company_Name, Industry, Location,
#        Job_Type, Job_Summary, Education_Requirements,
#        Experience_Level_Requirements, Experience_Length_Requirements,
#        Job_Description, Salary, Salary_Range, Date_Posted,
#        Day_Posted, Scrape_Time, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

# try:
#     # Execute the INSERT statement for each set of data
#     for row in rowed_data:
#         cursor.execute(insert_sql, row)
    
#     # Commit the changes to the database
#     cnxn.commit()
#     print('Data inserted successfully.')
# except pyodbc.Error as e:
#     print(f'Error inserting data: {str(e)}')
#     # Rollback the transaction in case of an error
#     cnxn.rollback()
# finally:
#     # Close the cursor and the database connection
#     cursor.close()
#     cnxn.close()